加载 MINIST 数据集

In [ ]:
import os
import mindspore.dataset as ds
import matplotlib.pyplot as plt

dataset_dir = "./MINIST/train"  # 数据集路径
# 从 mnist 数据集读取3张图片
mnist_dataset = ds.MnistDataset(dataset_dir, num_samples=3)
# 查看图像，设置图像大小
plt.figure(figsize=(8, 8))

# 打印 3 张子图
for i, dic in enumerate(mnist_dataset.create_dict_iterator(output_numpy=True)):
    plt.subplot(3, 3, i + 1)
    plt.imshow(dic["image"][:, :, 0])
    plt.axis("off")

plt.show()

自定义数据集

In [ ]:
import numpy as np
np.random.seed(58)

In [ ]:
class DatasetGenerator:
    # 实例化数据集对象时，__init__函数被调用，用户可以在此进行数据初始化等操作
    def __init__(self) -> None:
        self.data = np.random.sample((5, 2))
        self.label = np.random.sample((5, 1))

    # 定义数据集类的__getitem__函数，使其支持随机访问，能够根据给定的索引值index，获取数据集中的数据并返回。
    def __getitem__(self, index):
        return self.data[index], self.label[index]

    # 定义数据集类的__len__函数，返回数据集的样本数量。
    def __len__(self):
        return len(self.data)

In [ ]:
# 定义数据集类后，就可以通过GeneratorDataset接口按照用户定义的方式加载并访问数据集样本
dataset_generator = DatasetGenerator()
dataset = ds.GeneratorDataset(dataset_generator, ["data", "label"], shuffle=False)
# 通过 create_dict_iterator 方法获取数据
for data in dataset.create_dict_iterator():
    print("{}".format(data["data"]), "{}".format(data["label"]))

数据增强

In [ ]:
ds.config.set_seed(58)

# 随机打乱数据顷序，buffer_size表示数据集中进行shuffle操作的缓存区的大小。
dataset = dataset.shuffle(buffer_size=10)

# 对数据集进行分批，batch_size表示每组包含的数据个数，现设置每组包含2个数据。
dataset = dataset.batch(batch_size=2)

for data in dataset.create_dict_iterator():
    print("data: {}".format(data["data"]))
    print("label: {}".format(data["label"]))

In [ ]:
import matplotlib.pyplot as plt

from mindspore.dataset.vision import Inter
import mindspore.dataset.vision.c_transforms as c_vision

DATA_DIR = "./MINIST/train"
# 取出6个样本
mnist_dataset = ds.MnistDataset(DATA_DIR, num_samples=6, shuffle=False)
# 查看数据原图
mnist_it = mnist_dataset.create_dict_iterator()
data = next(mnist_it)
plt.imshow(data["image"].asnumpy().squeeze(), cmap=plt.cm.gray)
plt.title(data["label"].asnumpy(), fontsize=20)
plt.show()

In [ ]:
resize_op = c_vision.Resize((40, 40), interpolation=Inter.LINEAR)  # 定义resize操作
crop_op = c_vision.RandomCrop(28)
transforms_list = [resize_op, crop_op]
mnist_dataset = mnist_dataset.map(operations=transforms_list, input_columns=["image"])
mnist_dataset = mnist_dataset.create_dict_iterator()
data = next(mnist_dataset)
plt.imshow(data["image"].asnumpy().squeeze(), cmap=plt.cm.gray)
plt.title(data["label"].asnumpy(), fontsize=20)
plt.show()